Income data pulled from the 2012-2016 American Community Survey 5-Year Estimates at https://factfinder.census.gov/ . The search was performed for all 5-digit zip codes in Texas. Any zip code that did not have data for all 6 years were dropped. This removed 17 zip codes from our data set.  

In [1]:
import pandas as pd
import matplotlib as plt

In [31]:
income_11_df = pd.read_csv("household_income_11.csv", skiprows=1)
income_11_df = income_11_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_11_df = income_11_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_11_df = income_11_df[income_11_df["Median Income (dollars)"] != '-']
income_11_df["Year"] = 2011

income_12_df = pd.read_csv("household_income_12.csv", skiprows=1)
income_12_df = income_12_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_12_df = income_12_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_12_df = income_12_df[income_12_df["Median Income (dollars)"] != '-']
income_12_df["Year"] = 2012

income_13_df = pd.read_csv("household_income_13.csv", skiprows=1)
income_13_df = income_13_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_13_df = income_13_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_13_df = income_13_df[income_13_df["Median Income (dollars)"] != '-']
income_13_df["Year"] = 2013

income_14_df = pd.read_csv("household_income_14.csv", skiprows=1)
income_14_df = income_14_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_14_df = income_14_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_14_df = income_14_df[income_14_df["Median Income (dollars)"] != '-']
income_14_df["Year"] = 2014

income_15_df = pd.read_csv("household_income_15.csv", skiprows=1)
income_15_df = income_15_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_15_df = income_15_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_15_df = income_15_df[income_15_df["Median Income (dollars)"] != '-']
income_15_df["Year"] = 2015

income_16_df = pd.read_csv("household_income_16.csv", skiprows=1)
income_16_df = income_16_df[["Id2", "Households; Estimate; Total", "Households; Estimate; Median income (dollars)"]]
income_16_df = income_16_df.rename(index=str, columns={"Id2": "Zip Code", "Households; Estimate; Total": "Number of Households", "Households; Estimate; Median income (dollars)": "Median Income (dollars)"})
income_16_df = income_16_df[income_16_df["Median Income (dollars)"] != '-']
income_16_df["Year"] = 2016

income_11_12_df = income_11_df.append(income_12_df, ignore_index=True)
income_11_13_df = income_11_12_df.append(income_13_df, ignore_index=True)
income_11_14_df = income_11_13_df.append(income_14_df, ignore_index=True)
income_11_15_df = income_11_14_df.append(income_15_df, ignore_index=True)
income_11_16_df = income_11_15_df.append(income_16_df, ignore_index=True)

income_11_16_df.head(20)

,Zip Code,Number of Households,Median Income (dollars),Year
0,73949,605,42009,2011
1,75001,6607,55874,2011
2,75002,20393,91638,2011
3,75006,16534,57274,2011
4,75007,18284,78660,2011
5,75009,2744,76840,2011
6,75010,8157,74415,2011
7,75013,9317,123426,2011
8,75019,13641,106981,2011
9,75020,8945,41177,2011
